In [ ]:
"""
Author: Justin Perez (justin-perez@uiowa.edu)
Date: 2020-09-22

BEFORE COMMITTING CHANGES FROM THIS NOTEBOOK: edit -> clear all outputs
* otherwise all your outputs will get committed, which causes merge conflicts and noisy diffs
* if you forget before committing, just clear all outputs, `git add` it and `git commit --amend`

BEFORE RUNNING: review the values in cell 2

terminology:
    input length: count of 64-bit integers
    N: the set of input lengths to be tested
    n: represents a single value in N
    T: the set of '# of threads' values to be tested
    t: a single value in T
    mode: linux-style permission string, e.g., '-rw-rw-r--' (more info: https://www.comentum.com/images/permissions.jpg)
    

This notebook was tested with:
* hardware: 2020 M1 Macbook Pro (16 GB)
* python: 3.9.7 (distributed by homebrew)
    altair==4.1.0
    humanize==3.11.0
    jupyterlab==3.1.12
    pandas==1.3.3
* java: openjdk 16.0.2 (Temurin)

Tips:
    * RUN ALL: click the double triangle/fast-forward button (also restarts the kernel)
    * variables are scoped to the notebook, not individual cells
    * you can run cells in any order (although they may error if the code depends on state from a previous cell)
    * python notebooks keep their state between executions; to start from a clean state, restart the kernel
    * Ran w/ DEBUG=True and don't want to scroll for a year? View -> collapse all outputs
    * execute the currently selected cell: click the 'play'/triangle button or CMD+enter
                                                                        (prob ctrl + enter on linux/windows?)
    * jupyter notebooks run on ipython (an alternative to the standard python interpreter)
        docs:  https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#the-four-most-helpful-commands
        !abc      <-- executes 'abc' in your default shell
        x = !abc  <-- executes 'abc' in your default shell, captures stdout, assigns to x
        !abc $x   <-- executes 'abc <value of x>' in shell where x is a python variable
        %         <-- line 'magic' https://ipython.readthedocs.io/en/stable/interactive/magics.html#line-magics
        %%        <-- cell 'magic' https://ipython.readthedocs.io/en/stable/interactive/magics.html#cell-magics
"""
from datetime import timedelta   # qty of time e.g.: timedelta(seconds=90) == timedelta(minutes=1, seconds=30)
from itertools import product    # cross product
from pathlib import Path         # fairly similar to Java's Path class
import pandas as pd              # table-oriented data processing
import altair as alt             # charting
from stat import filemode        # for parsing file permissions (debugging)
import humanize                  # string formatting for humans, e.g. humanize.intcomma(1000) == "1,000"

# notebook normally only shows last element in a cell,
# use display to e.g., show a dataframe without putting it
# on the last line of a cell
from IPython.display import display 

# prevent strings in columns from getting truncated for being too long
pd.options.display.max_colwidth = None
# there's a similar pd option for max rows if you really wanna see every
# row in the data frames displayed in the debug output... can be slow to render

In [ ]:
"""VARIABLES YOU MIGHT WANT TO CHANGE

you should review these everytime you start the python server, or at least everytime
you rebase/merge with main. just so there are no surprises if someone commits changes.
"""

DEBUG = True  # controls output verbosity. True -> more verbose; False -> less verbose
FORCE_CREATE_INPUTS = False  # forces recreation of test input files even if they already exist

T = [2**i for i in range(8)]  # powers of 2 from 1 to 128
N = [10**i for i in range(4, 8)]  # powers of 10 from 10,000 to 10,000,000
combinations = list(product(N, T))  # every combination of (n in N, t in T), i.e. [(1, 10000), ... (128, 10000000)]
iterations = 2 # how many times to execute each combo

# True = load existing run time data from save path, False = make new data
skip_execution = load_existing_save = False

# force saving new run time data (only relevant when skip_execution/load_existing is False)
overwrite_existing_saved_timing = False


if DEBUG:    
    print(f"{T=}")  # what does f"{some_var=}" do? see: https://stackoverflow.com/a/59661963
    print(f"{N=}")
    humanized = [(humanize.intcomma(n), t) for n, t in combinations]
    display(pd.DataFrame(humanized, columns=["N", "T"]))

In [ ]:
"""stuff you hopefully don't need to change
these work on my machine (macos + zsh) but they may need to be tweaked to work on yours.

if something in here IS broken don't spend hours trying to fix it, message me (Justin) and ill help you fix
"""

# chart/table labels; use these constants for tab completion and consistency
THREADS = 'T (# of threads)'
INPUT = 'N (# of 64-bit integers)'
REAL = 'real time (seconds)'
USER = 'user execution time (cpu seconds)'
SYS = 'system overhead time (cpu seconds)'
LABELS = [INPUT, THREADS, REAL, USER, SYS]

# build up paths to things we need to run
base_dir = Path('..').resolve(strict=True) # strict=True to fail fast if the directories dont exist
input_dir = base_dir / "tmp"
class_path = (base_dir/"out"/"production"/"ParallelSort").resolve(strict=True)
output_path = (base_dir / 'sorted.bin').resolve(strict=False) # it's ok if input/output don't exist yet

# where to save the timing data from executing all those `combinations` `iterations` times,
# and whether to load an existing save file or collect a fresh set of data
timing_save_path = base_dir / 'observations.csv'


# figure out where java lives
java_command = !which java
assert len(java_command) == 1
java_command = java_command[0]

# define our runnable commands
enable_java_assertions = '-ea' if DEBUG else '' # java assertions add input/output validation but cost CPU cycles
base_command = f"{java_command} {'-ea' if DEBUG else ''} -cp {class_path}"
sort_command = f"{base_command} hw1.ParallelExternalLongSorter"
generate_command = f"{base_command} hw1.DataFileGenerator"
# this _should_ ensure we get consistent output between zsh and bash
# (zsh has a builtin `time` different from bash)
time_command = "TIMEFMT='%J  %U user %S system %P cpu %*E total' /usr/bin/time"

if DEBUG:
    print(f"""
        {LABELS=}
        {java_command=}
        {class_path=}
        {base_dir=}
        {output_path=}
        {sort_command=}
        {generate_command=}
    """)

In [ ]:
def generate(filename: Path, length:int):
    output = !$time_command $generate_command $filename $length
    return output
        
def sort(input_path:Path, output_path:Path, threads:int=0):
    cmd = f"{time_command} {sort_command} {input_path} {output_path} {threads or ''}"
    output = !$cmd
    return output

def get_time(command_output):
    """super hacky util function for parsing output of time command
    probably won't work for non-zsh shells
    note: ipython uses your default shell unless specified otherwise
    
    returns a dictionary: {"real": seconds, "user": seconds, "sys": seconds}
        seconds: float
        real: the "wall clock time"
        user: time executing the command
        sys: time spent on system overhead
    """
    # find the right line
    grep = command_output.grep('^\s*([\d\.]*) r')
    # get rid of extra white space
    fields = grep.fields()
    # we got a list of fields for each line of output, but there should only be one line of output
    assert len(fields) == 1
    fields = fields[0]
    # we're going to turn this into a dictionary, but the fields are in reverse order
    fields = fields[::-1]
    # now we have something like ["user", "1.3", "real", "2.8", "sys", "3.2"] and need to turn it into pairs
    it = iter(fields)
    pairs = zip(it, it)
    return dict(pairs)

if DEBUG:
    input_path = (base_dir / 'data.bin').resolve(strict=False)
    debug_outputs = [
        generate(input_path, 100),
        sort(input_path, output_path),
        sort(input_path, output_path, 1),
        sort(input_path, output_path, 2),
    ]
    print(debug_outputs)
    assert input_path.resolve(strict=True)
    assert output_path.resolve(strict=True)

In [ ]:
input_paths = {
    n: (base_dir / str(n)).with_suffix('.bin')
    for n in N
}

for n, p in input_paths.items():
    if FORCE_CREATE_INPUTS or not p.exists():
        generate(p, n)
    
if DEBUG:
    columns = ['n', 'path', 'mode', 'size']
    rows = []
    for n, p in input_paths.items():
        assert p.exists()
        assert p.is_file()
        st = p.stat()
        rows.append([
            humanize.scientific(n, precision=0),
            str(p.resolve(strict=True)),
            filemode(st.st_mode),
            humanize.naturalsize(st.st_size),
        ])
    df = pd.DataFrame(rows, columns=columns)
    display(df)

In [ ]:
def get_sort_time(n, t):
    output = sort(
        input_path=input_paths[n],
        output_path=output_path,
        threads=t,
    )
    return (get_time(output), output)
    

if DEBUG:
    n, t = combinations[0]
    timing, output = get_sort_time(n ,t)
    print(f"{timing=}")

In [ ]:
timings = {
    combo: [] for combo in combinations
}

if DEBUG:
    outputs = {
        combo: [] for combo in combinations
    }

for combo in combinations:
    if skip_execution:
        break
    n, t = combo
    for i in range(iterations):
        if DEBUG:
            print(f"starting iteration #{i} sorting {n=} 64-bit integers on {t=} threads")
        timing, output = get_sort_time(n, t)
        timings[combo].append(timing)
        if DEBUG:
            outputs[combo].append(output)

In [ ]:
if load_existing_save:
    df = pd.read_csv(timing_save_path)
else:    
    # time to flatten everything out
    rows = []

    for (n, t), time_dicts in timings.items():
        for time_dict in time_dicts:
            rows.append([
                n,
                t,
                # Altair doesn't like timedeltas
                #timedelta(seconds=float(time_dict['real'])),
                #timedelta(seconds=float(time_dict['user'])),
                #timedelta(seconds=float(time_dict['sys' ])),
                float(time_dict['real']),
                float(time_dict['user']),
                float(time_dict['sys' ]),
            ])
    if DEBUG:
        print(rows[:5])
        print('...')
        print(rows[-5:])
        
    df = pd.DataFrame(rows, columns=LABELS)
    if not timing_save_path.exists() or overwrite_existing_saved_timing:
        df.to_csv(timing_save_path)

if DEBUG:
    display(df)
    display(df.dtypes)
    ls_timing = !ls -lh $timing_save_path  #| grep $timing_save_path
    print('\n'.join(ls_timing))

In [ ]:
chart_types = [REAL, USER, SYS]
charts = dict()
for chart_type in chart_types:
    line = alt.Chart(df).mark_line().encode(
      x=THREADS, #
      y=f'mean({chart_type}):Q',
      color=f'{INPUT}:N'
    )

    band = alt.Chart(df).mark_errorband(extent='ci').encode(
        x=THREADS,
        y=alt.Y(f'{chart_type}:Q'),
        color=f'{INPUT}:N'
    )

    charts[chart_type] = line + band

In [ ]:
charts[REAL]

In [ ]:
charts[USER]

In [ ]:
charts[SYS]